In [26]:
import pandas as pd
# Import required libraries for Random Forest feature selection and XGBoost
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost as xgb
import warnings
# Import plotly for interactive visualizations
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
# Import PyTorch for neural networks
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [27]:
# Load and prepare the dataset
dfs = [pd.read_csv('D:\\AI-code\\Personal-AI-Projects\\PEMalwareClassification\\Dataset\\DLLs_Imported.csv'),
       pd.read_csv('D:\\AI-code\\Personal-AI-Projects\\PEMalwareClassification\\Dataset\\PE_Header.csv'),
       pd.read_csv('D:\\AI-code\\Personal-AI-Projects\\PEMalwareClassification\\Dataset\\PE_Section.csv')]

# Merge all datasets on SHA256
for df in dfs[1:]:
    dfs[0] = dfs[0].merge(df.drop(columns=['Type']), on='SHA256', how='left')
main_df = dfs[0]

# Explore the data structure
print("Dataset shape:", main_df.shape)
print("\nTarget variable distribution:")
print(main_df['Type'].value_counts())

# Separate features and target
X = main_df.drop(columns=['SHA256', 'Type'])
y = main_df['Type']

# Handle missing values - fill with 0 (appropriate for malware features)
X_clean = X.fillna(0)
print(f"Missing values after cleaning: {X_clean.isnull().sum().sum()}")

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(f"Encoded classes: {list(label_encoder.classes_)}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Dataset shape: (29498, 773)

Target variable distribution:
4    5076
1    5022
3    4957
2    4643
5    4224
6    3699
0    1877
Name: Type, dtype: int64
Missing values after cleaning: 0
Encoded classes: [0, 1, 2, 3, 4, 5, 6]
Training data shape: (23598, 771)
Testing data shape: (5900, 771)


In [28]:
# Feature Selection using Random Forest for Logistic Regression
print("Training Random Forest for feature importance...")

# Train Random Forest for feature selection
rf_feature_selector = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    max_depth=10
)

rf_feature_selector.fit(X_train, y_train)

# Get feature importances
feature_names = X.columns.tolist()
feature_importances = rf_feature_selector.feature_importances_

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

# Select top features for logistic regression (using more features for better performance)
n_features = 100  # Using top 100 features for better model performance
top_features = feature_importance_df.head(n_features)['feature'].tolist()

print(f"Selected top {n_features} features for Logistic Regression")
print("Top 10 selected features:")
for i, feature in enumerate(top_features[:10]):
    importance = feature_importance_df[feature_importance_df['feature'] == feature]['importance'].values[0]
    print(f"{i+1:2d}. {feature}: {importance:.4f}")

# Create reduced feature sets
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(f"\nOriginal feature set: {X_train.shape[1]} features")
print(f"Selected feature set: {X_train_selected.shape[1]} features")
print(f"Reduction: {(1 - X_train_selected.shape[1]/X_train.shape[1])*100:.1f}%")

Training Random Forest for feature importance...
Selected top 100 features for Logistic Regression
Top 10 selected features:
 1. TimeDateStamp: 0.0523
 2. text_Misc_VirtualSize: 0.0313
 3. data_VirtualAddress: 0.0285
 4. rsrc_PointerToRawData: 0.0283
 5. AddressOfEntryPoint: 0.0276
 6. CheckSum: 0.0275
 7. DllCharacteristics: 0.0275
 8. SizeOfCode: 0.0265
 9. SizeOfImage: 0.0261
10. text_SizeOfRawData: 0.0260

Original feature set: 771 features
Selected feature set: 100 features
Reduction: 87.0%


In [ ]:
# Neural Network Implementation for 6-Class PE Malware Classification
import torch.nn.functional as F
from sklearn.metrics import precision_recall_curve, roc_curve, auc, f1_score
from sklearn.preprocessing import label_binarize
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Define the Neural Network Architecture
class MalwareClassifierNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout_rate=0.3):
        super(MalwareClassifierNN, self).__init__()
        
        layers = []
        prev_size = input_size
        
        # Create hidden layers with batch normalization and dropout
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        # Output layer
        layers.append(nn.Linear(prev_size, num_classes))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Prepare data for PyTorch
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)
y_test_tensor = torch.LongTensor(y_test).to(device)

# Create DataLoaders
batch_size = 250
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model parameters
input_size = X_train_selected.shape[1]
hidden_sizes = [200, 100, 100, 100]  # Gradually decreasing hidden layer sizes
num_classes = len(np.unique(y_encoded))
dropout_rate = 0.2

print(f"Input size: {input_size}")
print(f"Number of classes: {num_classes}")
print(f"Hidden layer sizes: {hidden_sizes}")

# Initialize the model
model = MalwareClassifierNN(input_size, hidden_sizes, num_classes, dropout_rate).to(device)

# Calculate class weights for imbalanced dataset
class_counts = np.bincount(y_train)
class_weights = len(y_train) / (num_classes * class_counts)
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

# Loss function and optimizer with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Learning rate strategy: Start high and gradually decrease
initial_lr = 0.3  # Big learning rate
final_lr = 0.001  # Fixed final rate
num_epochs = 300

optimizer = torch.optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=1e-4)

# Learning rate scheduler - Step decay for continuous decrease
# Decrease learning rate by 0.001 every epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, 
    lr_lambda=lambda epoch: max((initial_lr - 0.01 * epoch) / initial_lr, final_lr / initial_lr)
)

print(f"Initial learning rate: {initial_lr}")
print(f"Final learning rate: {final_lr}")
print(f"Learning rate decrease: 0.003 per epoch")
print(f"Training for {num_epochs} epochs")
print(f"Model architecture:\n{model}")

Using device: cpu
Input size: 100
Number of classes: 7
Hidden layer sizes: [300, 200, 100, 50, 25]
Initial learning rate: 0.2
Final learning rate: 0.001
Learning rate decrease: 0.003 per epoch
Training for 300 epochs
Model architecture:
MalwareClassifierNN(
  (network): Sequential(
    (0): Linear(in_features=100, out_features=300, bias=True)
    (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=300, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=200, out_features=100, bias=True)
    (9): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=100, out_features=50, bias=True)
    (13): BatchNorm1d(50, eps=1e

In [30]:
# Training function with metrics tracking
def train_model():
    print("Clearing previous training state...")
    # Clear any previous outputs
    from IPython.display import clear_output
    
    model.train()
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    learning_rates = []
    
    # Early stopping parameters (set to None to disable)
    best_val_acc = 0.0
    patience = None  # Set to None to disable early stopping, or set to a number like 15 to enable
    patience_counter = 0
    
    print("Starting fresh training...")
    print("-" * 60)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for data, target in test_loader:
                outputs = model(data)
                loss = criterion(outputs, target)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += target.size(0)
                val_correct += (predicted == target).sum().item()
        
        # Calculate metrics
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store metrics
        train_losses.append(avg_train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)
        learning_rates.append(current_lr)
        
        # Update learning rate
        scheduler.step()
        
        # Early stopping check (only if patience is not None)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_malware_model.pth')
        elif patience is not None:  # Only increment counter if early stopping is enabled
            patience_counter += 1
        
        # Print progress every 10 epochs (without duplicates)
        if (epoch + 1) % 1 == 0 or epoch == 0:
            # Clear previous output every 10 epochs to avoid duplication
            if epoch > 0:
                clear_output(wait=True)
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%')
            print(f'Learning Rate: {current_lr:.6f}')
            print(f'Best Val Acc So Far: {best_val_acc:.2f}%')
            print('-' * 60)
        
        # Early stopping (only if patience is enabled)
        if patience is not None and patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
    # Load best model
    model.load_state_dict(torch.load('best_malware_model.pth'))
    
    return train_losses, train_accuracies, val_losses, val_accuracies, learning_rates

# Check if model was already trained to avoid re-training
if 'train_losses' not in globals() or len(train_losses) == 0:
    print("Training new model...")
    # Train the model
    train_losses, train_accuracies, val_losses, val_accuracies, learning_rates = train_model()
    print(f"\nTraining completed!")
    print(f"Best validation accuracy: {max(val_accuracies):.2f}%")
else:
    print("Model already trained. Current training metrics available.")
    print(f"Previous best validation accuracy: {max(val_accuracies):.2f}%")
    print("To retrain, delete the 'train_losses' variable or restart kernel.")

Epoch [47/300]
Train Loss: 0.9359, Train Acc: 65.65%
Val Loss: 0.8043, Val Acc: 72.85%
Learning Rate: 0.001000
Best Val Acc So Far: 73.29%
------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# Model Evaluation and Predictions
def evaluate_model():
    model.eval()
    test_predictions = []
    test_probs = []
    test_true = []
    
    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            test_predictions.extend(predicted.cpu().numpy())
            test_probs.extend(probabilities.cpu().numpy())
            test_true.extend(target.cpu().numpy())
    
    return np.array(test_predictions), np.array(test_probs), np.array(test_true)

# Get predictions and probabilities
test_predictions, test_probs, test_true = evaluate_model()

# Calculate overall accuracy
test_accuracy = accuracy_score(test_true, test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Generate classification report
class_names = [f'Class_{i}' for i in range(num_classes)]
print("\nClassification Report:")
print(classification_report(test_true, test_predictions, target_names=class_names))

# Calculate confusion matrix
cm = confusion_matrix(test_true, test_predictions)
print(f"\nConfusion Matrix shape: {cm.shape}")

# Binarize the output for multi-class metrics
y_test_binarized = label_binarize(test_true, classes=range(num_classes))

# Calculate ROC curves and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], test_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calculate macro-average ROC curve and AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Calculate Precision-Recall curves
precision = dict()
recall = dict()
pr_auc = dict()

for i in range(num_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_binarized[:, i], test_probs[:, i])
    pr_auc[i] = auc(recall[i], precision[i])

# Calculate macro F1-score
macro_f1 = f1_score(test_true, test_predictions, average='macro')
print(f"\nMacro F1-Score: {macro_f1:.4f}")

print(f"Macro ROC-AUC: {roc_auc['macro']:.4f}")
print(f"Individual class ROC-AUCs: {[f'{roc_auc[i]:.4f}' for i in range(num_classes)]}")
print(f"Individual class PR-AUCs: {[f'{pr_auc[i]:.4f}' for i in range(num_classes)]}")

Test Accuracy: 0.1392 (13.92%)

Classification Report:
              precision    recall  f1-score   support

     Class_0       0.11      0.01      0.01       375
     Class_1       0.12      0.07      0.09      1005
     Class_2       0.14      0.80      0.24       929
     Class_3       0.00      0.00      0.00       991
     Class_4       0.00      0.00      0.00      1015
     Class_5       0.12      0.00      0.00       845
     Class_6       0.00      0.00      0.00       740

    accuracy                           0.14      5900
   macro avg       0.07      0.13      0.05      5900
weighted avg       0.07      0.14      0.05      5900


Confusion Matrix shape: (7, 7)

Macro F1-Score: 0.0493
Macro ROC-AUC: 0.5241
Individual class ROC-AUCs: ['0.4829', '0.4864', '0.5096', '0.4896', '0.5465', '0.4067', '0.7463']
Individual class PR-AUCs: ['0.0614', '0.1598', '0.1752', '0.1671', '0.2350', '0.1103', '0.2033']


d:\AI-code\sklearn_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

d:\AI-code\sklearn_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

d:\AI-code\sklearn_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
# Plotly Visualizations

# 1. Training Progress Visualization (Loss and Learning Rate)
fig_training = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Training & Validation Loss', 'Training & Validation Accuracy', 
                   'Learning Rate Schedule', 'Macro F1-Score Progress'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

epochs = list(range(1, len(train_losses) + 1))

# Loss subplot
fig_training.add_trace(
    go.Scatter(x=epochs, y=train_losses, name='Train Loss', line=dict(color='blue')),
    row=1, col=1
)
fig_training.add_trace(
    go.Scatter(x=epochs, y=val_losses, name='Validation Loss', line=dict(color='red')),
    row=1, col=1
)

# Accuracy subplot
fig_training.add_trace(
    go.Scatter(x=epochs, y=train_accuracies, name='Train Accuracy', line=dict(color='green')),
    row=1, col=2
)
fig_training.add_trace(
    go.Scatter(x=epochs, y=val_accuracies, name='Validation Accuracy', line=dict(color='orange')),
    row=1, col=2
)

# Learning rate subplot
fig_training.add_trace(
    go.Scatter(x=epochs, y=learning_rates, name='Learning Rate', line=dict(color='purple')),
    row=2, col=1
)

# Calculate F1 scores for each epoch (approximation using validation accuracy as proxy)
f1_progress = [acc/100 * 0.8 + 0.1 for acc in val_accuracies]  # Approximate F1 from accuracy
fig_training.add_trace(
    go.Scatter(x=epochs, y=f1_progress, name='Approx. Macro F1', line=dict(color='brown')),
    row=2, col=2
)

fig_training.update_layout(
    title_text="Neural Network Training Progress",
    height=800,
    showlegend=True
)

fig_training.update_xaxes(title_text="Epoch", row=1, col=1)
fig_training.update_xaxes(title_text="Epoch", row=1, col=2)
fig_training.update_xaxes(title_text="Epoch", row=2, col=1)
fig_training.update_xaxes(title_text="Epoch", row=2, col=2)

fig_training.update_yaxes(title_text="Loss", row=1, col=1)
fig_training.update_yaxes(title_text="Accuracy (%)", row=1, col=2)
fig_training.update_yaxes(title_text="Learning Rate", row=2, col=1)
fig_training.update_yaxes(title_text="F1-Score", row=2, col=2)

fig_training.show()

print("Training progress visualization created!")

Training progress visualization created!


In [ ]:
# 2. Interactive Heatmap Confusion Matrix
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=[f'Predicted Class {i}' for i in range(num_classes)],
    y=[f'True Class {i}' for i in range(num_classes)],
    colorscale='Blues',
    text=cm,
    texttemplate="%{text}",
    textfont={"size": 12},
    hovertemplate='True Class: %{y}<br>Predicted Class: %{x}<br>Count: %{z}<extra></extra>'
))

# Add annotations for better readability
annotations = []
for i in range(num_classes):
    for j in range(num_classes):
        annotations.append(
            dict(
                x=j, y=i,
                text=str(cm[i][j]),
                showarrow=False,
                font=dict(color="white" if cm[i][j] > cm.max()/2 else "black", size=14)
            )
        )

fig_cm.update_layout(
    title='Confusion Matrix Heatmap - 6-Class PE Malware Classification',
    xaxis_title='Predicted Class',
    yaxis_title='True Class',
    width=700,
    height=600,
    annotations=annotations
)

fig_cm.show()

# Calculate and display per-class metrics
print("Per-Class Performance Metrics:")
print("-" * 50)
for i in range(num_classes):
    true_positives = cm[i, i]
    false_positives = cm[:, i].sum() - true_positives
    false_negatives = cm[i, :].sum() - true_positives
    
    precision_val = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall_val = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_val = 2 * (precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0
    
    class_accuracy = true_positives / cm[i, :].sum() if cm[i, :].sum() > 0 else 0
    
    print(f"Class {i}:")
    print(f"  Precision: {precision_val:.4f}")
    print(f"  Recall: {recall_val:.4f}")
    print(f"  F1-Score: {f1_val:.4f}")
    print(f"  Accuracy: {class_accuracy:.4f}")
    print()

Per-Class Performance Metrics:
--------------------------------------------------
Class 0:
  Precision: 0.1053
  Recall: 0.0053
  F1-Score: 0.0102
  Accuracy: 0.0053

Class 1:
  Precision: 0.1161
  Recall: 0.0746
  F1-Score: 0.0909
  Accuracy: 0.0746

Class 2:
  Precision: 0.1422
  Recall: 0.7998
  F1-Score: 0.2415
  Accuracy: 0.7998

Class 3:
  Precision: 0.0000
  Recall: 0.0000
  F1-Score: 0.0000
  Accuracy: 0.0000

Class 4:
  Precision: 0.0000
  Recall: 0.0000
  F1-Score: 0.0000
  Accuracy: 0.0000

Class 5:
  Precision: 0.1250
  Recall: 0.0012
  F1-Score: 0.0023
  Accuracy: 0.0012

Class 6:
  Precision: 0.0000
  Recall: 0.0000
  F1-Score: 0.0000
  Accuracy: 0.0000



In [ ]:
# 3. Macro ROC Curve Visualization
fig_roc = go.Figure()

# Plot ROC curve for each class
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
for i in range(num_classes):
    fig_roc.add_trace(go.Scatter(
        x=fpr[i], 
        y=tpr[i],
        mode='lines',
        name=f'Class {i} (AUC = {roc_auc[i]:.3f})',
        line=dict(color=colors[i % len(colors)], width=2)
    ))

# Plot macro-average ROC curve
fig_roc.add_trace(go.Scatter(
    x=fpr["macro"], 
    y=tpr["macro"],
    mode='lines',
    name=f'Macro-Average (AUC = {roc_auc["macro"]:.3f})',
    line=dict(color='black', width=3, dash='dash')
))

# Add diagonal line (random classifier)
fig_roc.add_trace(go.Scatter(
    x=[0, 1], 
    y=[0, 1],
    mode='lines',
    name='Random Classifier',
    line=dict(color='gray', width=1, dash='dot')
))

fig_roc.update_layout(
    title='Macro ROC Curves - 6-Class PE Malware Classification',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=800,
    height=600,
    legend=dict(x=0.6, y=0.3)
)

fig_roc.update_xaxes(range=[0, 1])
fig_roc.update_yaxes(range=[0, 1])

fig_roc.show()

print(f"Macro ROC-AUC Score: {roc_auc['macro']:.4f}")

Macro ROC-AUC Score: 0.5241


In [ ]:
# 4. Precision-Recall Curves
fig_pr = go.Figure()

# Plot PR curve for each class
for i in range(num_classes):
    fig_pr.add_trace(go.Scatter(
        x=recall[i], 
        y=precision[i],
        mode='lines',
        name=f'Class {i} (AUC = {pr_auc[i]:.3f})',
        line=dict(color=colors[i % len(colors)], width=2)
    ))

# Add baseline (random classifier for balanced dataset)
baseline = y_test_binarized.sum(axis=0) / len(y_test_binarized)
for i in range(num_classes):
    fig_pr.add_hline(
        y=baseline[i], 
        line_dash="dot", 
        line_color="gray",
        annotation_text=f"Random Class {i}",
        annotation_position="right"
    )

fig_pr.update_layout(
    title='Precision-Recall Curves - 6-Class PE Malware Classification',
    xaxis_title='Recall',
    yaxis_title='Precision',
    width=800,
    height=600,
    legend=dict(x=0.6, y=0.9)
)

fig_pr.update_xaxes(range=[0, 1])
fig_pr.update_yaxes(range=[0, 1])

fig_pr.show()

# Calculate and display average PR-AUC
avg_pr_auc = np.mean([pr_auc[i] for i in range(num_classes)])
print(f"Average PR-AUC Score: {avg_pr_auc:.4f}")
print("Individual PR-AUC Scores:")
for i in range(num_classes):
    print(f"  Class {i}: {pr_auc[i]:.4f}")

Average PR-AUC Score: 0.1589
Individual PR-AUC Scores:
  Class 0: 0.0614
  Class 1: 0.1598
  Class 2: 0.1752
  Class 3: 0.1671
  Class 4: 0.2350
  Class 5: 0.1103
  Class 6: 0.2033


In [ ]:
# 5. Comprehensive F1-Score Visualization
f1_scores_final = []
precisions_final = []
recalls_final = []

for i in range(num_classes):
    true_positives = cm[i, i]
    false_positives = cm[:, i].sum() - true_positives
    false_negatives = cm[i, :].sum() - true_positives
    
    precision_val = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall_val = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_val = 2 * (precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0
    
    f1_scores_final.append(f1_val)
    precisions_final.append(precision_val)
    recalls_final.append(recall_val)

# Create F1-Score bar chart
fig_f1 = go.Figure()

fig_f1.add_trace(go.Bar(
    x=[f'Class {i}' for i in range(num_classes)],
    y=f1_scores_final,
    name='F1-Score',
    marker_color='lightblue',
    text=[f'{score:.3f}' for score in f1_scores_final],
    textposition='auto'
))

# Add macro F1-score line
fig_f1.add_hline(
    y=macro_f1, 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"Macro F1: {macro_f1:.3f}",
    annotation_position="top right"
)

fig_f1.update_layout(
    title='F1-Scores by Class - 6-Class PE Malware Classification',
    xaxis_title='Class',
    yaxis_title='F1-Score',
    width=800,
    height=500,
    yaxis=dict(range=[0, 1])
)

fig_f1.show()

# Create comprehensive metrics comparison
metrics_comparison = go.Figure()

x_pos = np.arange(num_classes)
width = 0.25

metrics_comparison.add_trace(go.Bar(
    name='Precision',
    x=[f'Class {i}' for i in range(num_classes)],
    y=precisions_final,
    marker_color='lightcoral'
))

metrics_comparison.add_trace(go.Bar(
    name='Recall',
    x=[f'Class {i}' for i in range(num_classes)],
    y=recalls_final,
    marker_color='lightgreen'
))

metrics_comparison.add_trace(go.Bar(
    name='F1-Score',
    x=[f'Class {i}' for i in range(num_classes)],
    y=f1_scores_final,
    marker_color='lightblue'
))

metrics_comparison.update_layout(
    title='Precision, Recall, and F1-Score Comparison by Class',
    xaxis_title='Class',
    yaxis_title='Score',
    barmode='group',
    width=900,
    height=500,
    yaxis=dict(range=[0, 1])
)

metrics_comparison.show()

print("All visualizations completed!")
print("\nFinal Model Summary:")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Macro F1-Score: {macro_f1:.4f}")
print(f"Macro ROC-AUC: {roc_auc['macro']:.4f}")
print(f"Average PR-AUC: {avg_pr_auc:.4f}")
print(f"Training completed in {len(train_losses)} epochs")

All visualizations completed!

Final Model Summary:
Test Accuracy: 13.92%
Macro F1-Score: 0.0493
Macro ROC-AUC: 0.5241
Average PR-AUC: 0.1589
Training completed in 50 epochs
